In [1]:
import os
import torch


os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from transformers import (
    GPTJForCausalLM,
    GPT2Tokenizer,
    LlamaForCausalLM, 
    LlamaTokenizer,
)

from transformers.trainer_utils import get_last_checkpoint
import transformers
import re
import pandas as pd
import torch
from tqdm import tqdm

# model_path = "/data/jzheng36/Fine-Tune-Llama2-/llama2_ft_7B-lr=2e-4"
model_path = "/data/shared/llama2/llama/7B-Chat/"
# model_path = "philschmid/llama-2-7b-instruction-generator"

# model_path = "/data/jzheng36/model/instruct_gpt4chan_3epoch"
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# # GPT-4chan
# model = GPTJForCausalLM.from_pretrained(
#     get_last_checkpoint(model_path) # 3, 10
# ).to(device)
# tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-j-6B")

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(model_path)

pipeline = transformers.pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=8,
    device=device,
   
)

cuda

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /data/installation/anaconda3/envs/lora/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...


/data/installation/anaconda3/envs/lora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /data/installation/anaconda3/envs/lora did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/data/installation/anaconda3/envs/lora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacty of 31.74 GiB of which 165.62 MiB is free. Process 93755 has 30.84 GiB memory in use. Including non-PyTorch memory, this process has 750.00 MiB memory in use. Of the allocated memory 0 bytes is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
import random
import pathlib

from termcolor import (
    cprint,
    colored
)

def extract_category(text):
    text = re.sub(r'\s*-\s*description:\s*', '', text)
    patterns = [
        r'Category: \[(.*?)\]',
        r"\\n \\n Category:(.*?) \\n \\n",
        r"\n \n Category:(.*?) \n",
        r'\n Category:(.*?)\"',
        r'\n Category:(.*?)$'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1).strip()

    return text.replace('\n', '')

In [26]:
df = pd.read_csv('guidelines.csv', sep = '\t')
with open('/data/jzheng36/auto_annotation/prompt_zeroshot.txt', 'r') as file:
    content = file.read()

In [27]:
# def classify_by_llama(text):
#     examples = [
#         {
#             "label": "Non-hate",
#             "index": 41,
#             "description": "Other non-hateful sentences."
#         },
#     ]
#     for i in range(41):
#         example = {
#                 "label": df["guideline"][i],
#                 "index": df["index"][i],
#                 "text": df["examples"][i],
#                 "description": df["description"][i]
#             }
#         examples.append(example)


#     prefix = "Classify the sentence into one of the labels and response by label only:\n"
#     prompt_template = "\n\n".join("- label: {}\n- description: {}".format(example["label"], example["description"]) 
#                               for example in examples[1:])
    
#     suffix = "\n\n- text: {}\n- label:".format(text)


#     text_input = prefix + prompt_template + suffix
#     return text_input
    
# text = "Donald Trump is smelly pig."
# text_input = classify_by_llama(text)

In [28]:
def classify_by_llama(text):
    prefix = "Classify the sentence into one of the labels and response by label only:\n"  
    suffix = "\n\n- text: {}\n- label:".format(text)

    text_input = prefix + content + suffix
    return text_input
    
text = "Donald Trump is smelly pig."
text_input = classify_by_llama(text)

In [29]:
print(text_input)

Classify the sentence into one of the labels and response by label only:
Classify the sentence into one of the labels and response by label only:
- label: filth
- description: Dehumanizing speech: Filth (including but not limited to: dirt, grime)

- label: violent speech
- description: Violent speech or support in written or visual form

- label: insects
- description: Dehumanizing speech: Insects (including but not limited to: cockroaches, locusts)

- label: bacteria
- description: Dehumanizing speech: Bacteria, viruses, or microbes

- label: disease
- description: Dehumanizing speech: Disease (including but not limited to: cancer, sexually transmitted diseases)

- label: dehumanization animal
- description: Dehumanizing speech: Animals in general or specific types of animals that are culturally perceived as intellectually or physically inferior (including but not limited to: Black people and apes or ape-like

- label: feces
- description: Dehumanizing speech: Feces (including but not

In [30]:
response = pipeline(text_input)
continuation = response[0]['generated_text'].replace(text_input, "")
    
print(colored(text_input, "red") + colored(continuation, "green"))

/data/installation/anaconda3/envs/lora/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Classify the sentence into one of the labels and response by label only:
Classify the sentence into one of the labels and response by label only:
- label: filth
- description: Dehumanizing speech: Filth (including but not limited to: dirt, grime)

- label: violent speech
- description: Violent speech or support in written or visual form

- label: insects
- description: Dehumanizing speech: Insects (including but not limited to: cockroaches, locusts)

- label: bacteria
- description: Dehumanizing speech: Bacteria, viruses, or microbes

- label: disease
- description: Dehumanizing speech: Disease (including but not limited to: cancer, sexually transmitted diseases)

- label: dehumanization animal
- description: Dehumanizing speech: Animals in general or specific types of animals that are culturally perceived as intellectually or physically inferior (including but not limited to: Black people and apes or ape-like

- label: feces
- description: Dehumanizing speech: Feces (including but not

In [31]:
extract_category(continuation)

' dehumanization animal'

In [32]:
df = pd.read_csv("/data/jzheng36/auto_annotation/test_dataset.csv", sep = '\t')

labels = []
for sentence in tqdm(df["text"]):
    text_input = classify_by_llama(sentence)
    response = pipeline(text_input)
    label = response[0]['generated_text'].replace(text_input, "")
    label = extract_category(label)
    print(label)
    labels.append(label)
    
df["prediction"] = labels
df.to_csv("/data/jzheng36/auto_annotation/test_with_llama2.csv", sep="\t", index=False)


  0%|          | 0/123 [00:00<?, ?it/s]/data/installation/anaconda3/envs/lora/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  1%|          | 1/123 [00:02<05:51,  2.88s/it]

 contempt self admission intoler


  2%|▏         | 2/123 [00:05<05:45,  2.86s/it]

 contempt self admission intoler


  2%|▏         | 3/123 [00:08<05:42,  2.85s/it]

...

 filthDehuman


 99%|█████████▉| 122/123 [05:50<00:02,  2.89s/it]

 violent speechViol


100%|██████████| 123/123 [05:53<00:00,  2.87s/it]

 filthDehuman
